In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np

class Modelo:
    def __init__(self):
        self.df = None
        self.models = {}
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.scaler = None

    def CarregarDataset(self, path):
        """Carrega o conjunto de dados a partir de um arquivo CSV."""
        names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
        self.df = pd.read_csv(path, names=names)

    def TratamentoDeDados(self):
        """Realiza o pré-processamento dos dados carregados."""
        # Verificação de valores NaN e análise inicial
        self.df.info()
        print(f"\n{self.df.nunique()}") 
        print(f"\n{self.df.Species.value_counts()}\n")

        # Converte as espécies em valores numéricos
        le = LabelEncoder()
        self.df['Species'] = le.fit_transform(self.df['Species'])

        # Separa características (X) e espécies (y)
        X = self.df.drop('Species', axis=1)
        y = self.df['Species']

        # Normalização das características
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)

        # Divide em treino e teste
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X_scaled, y, test_size=0.3, random_state=42, stratify=y  
        )

    def Treinamento(self):
        """Treina modelos de machine learning com validação cruzada."""
        # Modelos a serem treinados
        models_to_train = {
            'SVM': SVC(kernel='linear', random_state=42),
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=200)
        }

        for name, model in models_to_train.items():
            # Validação cruzada
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            scores = cross_val_score(model, self.X_train, self.y_train, cv=skf, scoring='accuracy')
            print(f"Validação cruzada para {name}: {scores}, Média: {np.mean(scores):.2f}")

            # Treinamento no conjunto completo de treino
            model.fit(self.X_train, self.y_train)
            self.models[name] = model

    def Teste(self):
        """Avalia o desempenho dos modelos treinados."""
        resultados = {}
        for name, model in self.models.items():
            y_pred = model.predict(self.X_test)
            acc = accuracy_score(self.y_test, y_pred)
            resultados[name] = acc
        return resultados

    def Train(self):
        """Processo de treinamento e avaliação."""
        self.CarregarDataset("iris.data") # Caminho local para o arquivo iris.data
        self.TratamentoDeDados()
        self.Treinamento()
        resultados = self.Teste()

        # Exibe a acurácia de cada modelo baseado no conjunto de teste
        for model_name, accuracy in resultados.items():
            print(f"Modelo: {model_name}, Acurácia no conjunto de teste: {accuracy:.2f}")

# Execução dos modelos
modelo = Modelo()
modelo.Train()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB

SepalLengthCm    35
SepalWidthCm     23
PetalLengthCm    43
PetalWidthCm     22
Species           3
dtype: int64

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

Validação cruzada para SVM: [0.9047619  0.95238095 1.         1.         0.95238095], Média: 0.96
Validação cruzada para Logistic Regression: [0.95238095 1.         1.         1.         0.95238095], Média: 0.98
Modelo: SVM, Acurácia no conjunto de teste: 0.91
Modelo: Logistic Regression, Acurácia no conjunto de teste:

Conlusão:

Após a execução do código, foram obtidos valores para a validação cruzada e a acurácia no conjunto de teste dos dois modelos utilizados: SVM e Regressão Logística (Logistic Regression). Para a validação cruzada, cada modelo recebeu respectivamente:
[0.9047619  0.95238095 1.         1.         0.95238095], Média: 0.96;
[0.95238095 1.         1.         1.         0.95238095], Média: 0.98.
Ainda, a acurácia no conjunto de teste foi de 0.91 para ambos modelos.

Dessa forma, é possível concluir que a precisão e eficiência dos dois modelos foram significativamente similares, tendo apenas uma diferença de 0.02 na validação cruzada. Isso demonstra que, nessa escala de dados, o resultado de um treinamento com a mesma fonte gerará um resultado parecido, de modo que é possível escolher qualquer um dos dois tipos sem com que haja uma diferença nítida. Assim como é perceptível o grande valor da precisão obtida, provando a eficiência desses modelos nessa ocasião.

